# Análise de Dados de Crédito 

## O objetivo desta análise é entender quais variáveis podem indicar uma probabilidade maior de default de crédito visando atuar de forma preventiva.
- Tratar os dados da base de crédito
- Analisar os dados Qualitativos
- Analisar os dados Quantitativos 
- Analisar possíveis correlações

In [36]:
import os

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [38]:
df = pd.read_csv('/kaggle/input/credit-dataset/Python_M10_support material.csv', na_values = ['','none','null','NaN','N/A','na'])

## 1. Exploração do dataset

In [39]:
df.shape

In [40]:
df.dtypes

In [41]:
df.select_dtypes('object').describe().transpose()

In [42]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

In [43]:
df.isna().any()

In [44]:
df.isna().sum()

# 2. Tratamento dos dados

### Tratar as colunas limite_credito e valor_transacoes_12m de object para float

In [45]:
df[['limite_credito','valor_transacoes_12m']].head()

In [46]:
limite_credito = df['limite_credito'].head()
limite_credito = limite_credito.to_list()
print(limite_credito)
print(type(limite_credito))

In [47]:
fn = lambda valor: float(valor.replace('.','').replace(',','.'))

valor_limpo =  list(map(fn, limite_credito))

print(valor_limpo)

In [48]:
#uma vez testada a função para transformação dos objetos em floats, vou aplicar nas colunas de interesse do dataframe.

df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(fn)
df['limite_credito'] = df['limite_credito'].apply(fn)

In [49]:
df.dtypes

In [50]:
df.select_dtypes('object').describe().transpose()

In [51]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

# Tratando dados N/A

In [52]:
# distribuição original de default

df_total, _ = df.shape
df_adimplente, _ = df[df['default']==0].shape
df_inadimplente, _ = df[df['default']==1].shape

In [53]:
df.dropna(inplace=True)

In [54]:
df.shape

In [55]:
# distribuição dos dados antes e depois da limpeza dos dados faltantes.

df_total_novo, _ = df.shape
df_adimplente_novo, _ = df[df['default']==0].shape
df_inadimplente_novo, _ = df[df['default']==1].shape

print(f"a proporção de clientes adimplentes é {round(df_adimplente/df_total*100,2)}%")
print(f"a proporção de clientes inadimplentes é {round(df_inadimplente/df_total*100,2)}%")
print("")
print(f"a proporção de clientes adimplentes é {round(df_adimplente_novo/df_total_novo*100,2)}%")
print(f"a proporção de clientes adimplentes é {round(df_inadimplente_novo/df_total_novo*100,2)}%")

Uma vez que os dados foram tratados e a distribuição não foi afetada de forma significativa posso seguir para a visualização e análise dos dados

## 3. Visualização dos dados

# Categóricos

In [56]:
sns.set_style('whitegrid')

#avaliar o comportamento das variáveis separado por base total, default e non default.

df_adimplente = df[df['default']==0]
df_inadimplente = df[df['default']==1]

In [57]:
# Primeiramente vou retomar as variáveis categóricas e definir uma função para plotagem e análise.

df.select_dtypes('object').head()

In [58]:
def viz_categorica(coluna, titulos):
    
    eixo = 0
    max_y = 0
    ma = df.select_dtypes('object').describe()[coluna]['freq']*1.1
    
    figura, eixos = plt.subplots(1,3, figsize=(20,5), sharex = True)
    
    for dataframe in [df,df_adimplente,df_inadimplente]:
        df_to_plot = dataframe[coluna].value_counts().to_frame()
        df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace= True)
        df_to_plot[coluna] = df_to_plot.index
        df_to_plot.sort_values(by=[coluna], inplace=True)
    
        f = sns.barplot(x=df_to_plot[coluna],y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
        f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequencia Absoluta')
        f.set_xticklabels(labels=f.get_xticklabels(),rotation=90)

        _, max_y_f = f.get_ylim()
        max_y = max_y_f if max_y_f > max_y else max_y
        f.set(ylim=(0,max_y))
    
        eixo += 1
    
    figura.show()


In [59]:
# Análise da distribuição por Escolaridade

coluna = 'escolaridade'
titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes', 'Escolaridade dos Clientes Inadimplentes']

_ = viz_categorica(coluna, titulos)

In [60]:
# Análise da distribuição por Salário Anual

coluna = 'salario_anual'
titulos = ['Salário Anual dos Clientes', 'Salário Anual dos Clientes Adimplentes', 'Salário Anual dos Clientes Inadimplentes']

_ = viz_categorica(coluna, titulos)

In [61]:
# Análise da distribuição por Tipo de cartão

coluna = 'tipo_cartao'
titulos = ['Tipo de cartão dos Clientes', 'Tipo de cartão dos Clientes Adimplentes', 'Tipo de cartão dos Clientes Inadimplentes']

_ = viz_categorica(coluna, titulos)

As visualizações dos dados categóricos não mostraram nenhuma diferença relevante na distibuição entre a base e as categorias de default!

# Numéricos

In [62]:
df.select_dtypes('number').drop(['id','default'],axis=1).head()

In [63]:
def viz_numerica(coluna, titulos):
    
    eixo = 0
    max_y = 0
    figura, eixos = plt.subplots(1,3, figsize=(20,5), sharex = True)
    
    for dataframe in [df,df_adimplente,df_inadimplente]:
    
        f = sns.histplot(x=coluna,data=dataframe,stat='count', ax=eixos[eixo])
        f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequencia Absoluta')
        

        _, max_y_f = f.get_ylim()
        max_y = max_y_f if max_y_f > max_y else max_y
        f.set(ylim=(0,max_y))
    
        eixo += 1
    
    figura.show()


In [64]:
# Análise da distribuição da Quantidade de produtos

coluna = 'qtd_produtos'
titulos = ['Quantidade de Produtos dos Clientes', 'Quantidade de Produtos dos Clientes Adimplentes', 'Quantidade de Produtos dos Clientes Inadimplentes']

_ = viz_numerica(coluna, titulos)

In [65]:
# Análise da distribuição por Quantidade de Transações em 12m

coluna = 'qtd_transacoes_12m'
titulos = ['Quantidade de Transações dos Clientes', 'Quantidade de Transações dos Clientes Adimplentes', 'Quantidade de Transações dos Clientes Inadimplentes']

_ = viz_numerica(coluna, titulos)

In [66]:
# Análise da distribuição por Valor das Transações em 12m

coluna = 'valor_transacoes_12m'
titulos = ['Valor de Transação 12m dos Clientes', 'Valor de Transação 12m dos Clientes Adimplentes', 'Valor de Transação 12m dos Clientes Inadimplentes']

_ = viz_numerica(coluna, titulos)

# Analisando correlações

In [67]:
f = sns.relplot(x='valor_transacoes_12m',y='qtd_transacoes_12m', data=df, hue='default')
_ = f.set(
        title='Relação entre Valor e Quantidade transicionada no último ano',
        xlabel= 'Valor das Transações no Último Ano',
        ylabel= 'Quantidade das Transações no Último Ano')

In [68]:
f = sns.relplot(y='valor_transacoes_12m',x='qtd_produtos', data=df, hue='default')
_ = f.set(
        title='Relação entre Valor e Quantidade transicionada no último ano',
        ylabel= 'Valor das Transações no Último Ano',
        xlabel= 'Quantidade de Produtos Utilizados pelo Cliente')

In [69]:
f = sns.relplot(y='qtd_transacoes_12m',x='qtd_produtos', data=df, hue='default')
_ = f.set(
        title='Relação entre Valor e Quantidade transicionada no último ano',
        ylabel= 'Quantidade das Transações no Último Ano',
        xlabel= 'Quantidade de Produtos Utilizados pelo Cliente')

# Insights

Após tratar os dados das variáveis que possivelmente poderiam explicar o default de crédito as análises trouxeram alguns clusters.

- Apesar de um leve indício de mudança na distribuição de default quando analisada a quantidade de produtos utilizadas pelo cliente, ao correlacionar essa variável com a quantidade de transações em 12m e valor das transações em 12m não foi possível confirmar nenhum viés.
- Uma concentração de default grande entre clientes que realizaram de **30 a 50 transações em 12 meses** com **valor total entre 1500 e 3000.**
- Uma concentração um pouco menor entre clientes que realizaram de **60 a 80 transações em 12 meses** com **valor total entre 7500 e 10000.**

Esses clusters identificados devem ser acompanhados de forma mais próxima visando minimizar a probabilidade de default.